In [ ]:
!pip install mediapipe

In [27]:
import cv2
import pandas as pd
import mediapipe as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
import math

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Rutas
Definición de las rutas de input y output

In [4]:
video_path = '/content/drive/MyDrive/Física Muscle-up/lat_tincho.mov'
output_video_path = '/content/drive/MyDrive/Física Muscle-up/tracked_video.mp4'
output_csv_path = '/content/drive/MyDrive/Física Muscle-up/pose_data.csv'

#Input usuario
peso persona

In [5]:
peso_persona = 65 #kg
altura_persona = 176 #cm
peso_pesa = 140 #kg

#Crear columnas del dataframe

Procesa el video y almacena los datos de las poses. Define las columnas para un DataFrame donde se guardarán las coordenadas de las articulaciones detectadas en cada cuadro del video.

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(video_path)

video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_fps = cap.get(cv2.CAP_PROP_FPS)
tiempo_por_frame = 1/video_fps

print("Width:", video_width)
print("Height:", video_height)
print("Frames per second:", video_fps)
print("tiempo_por_frame:", tiempo_por_frame)

# Defino cuales son las articulaciones que me interesa estudiar
articulaciones = [
    mp_pose.PoseLandmark.LEFT_WRIST,
    mp_pose.PoseLandmark.RIGHT_WRIST,
    mp_pose.PoseLandmark.LEFT_HIP,
    mp_pose.PoseLandmark.RIGHT_HIP,
    mp_pose.PoseLandmark.LEFT_KNEE,
    mp_pose.PoseLandmark.RIGHT_KNEE,
    mp_pose.PoseLandmark.LEFT_ANKLE,
    mp_pose.PoseLandmark.RIGHT_ANKLE
]

columns = ['frame_number']
for landmark in articulaciones:
    columns.append(landmark.name + '_X')
    columns.append(landmark.name + '_Y')
    columns.append(landmark.name + '_Z')
columns.append("Tiempo")
columns.append("VelocidadAngular")
columns.append("Velocidad(Rodilla)_X")
columns.append("Velocidad(Rodilla)_Y")
columns.append("Velocidad(Cadera)_X")
columns.append("Velocidad(Cadera)_Y")
columns.append("Aceleracion(Rodilla)_X")
columns.append("Aceleracion(Rodilla)_Y")
columns.append("Aceleracion(Cadera)_X")
columns.append("Aceleracion(Cadera)_Y")
columns.append("Torque(Rodilla)")
columns.append("Torque(Cadera)")

df_completo = pd.DataFrame(columns=columns)

pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), video_fps, (int(cap.get(3)), int(cap.get(4))))

Width: 2160
Height: 3840
Frames per second: 60.0
tiempo_por_frame: 0.016666666666666666


# Función para calcular ángulo
Esta función toma tres puntos y calcula el ángulo formado por los segmentos de línea que conectan estos puntos.

In [7]:
def calculate_angle(a, b, c):
  #articulaciones convertidas en arreglos para poder usar sus datos en las coordenadas X e Y
  art_1 = np.array(a)
  art_2 = np.array(b)
  art_3 = np.array(c)

  radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])

  #El ángulo calculado se convierte de radianes a grados y se toma el valor absoluto --> Grados = Radianes * 180 / π
  angle = np.abs(radians*180.0/np.pi)

  #Se normaliza el ángulo calculado para asegurarse de que esté en el rango de 0 a 180 grados.
  if angle>180.0:
    angle = 360-angle

  return angle

#Función extraer posiciones
Toma un conjunto de landmarks y una lista de nombres de articulaciones. Luego, itera sobre cada nombre de parte del cuerpo proporcionado y extrae las coordenadas (x, y, z) del landmark correspondiente a esa parte del cuerpo. Estas coordenadas se agrupan en una lista y se devuelve una lista que contiene todas estas listas de coordenadas.

In [8]:
# formato output:([[x1, y1, z1], [x2, y2, z2], [x3, y3, z3]]).
def extraer_posiciones(df, frame_number, *articulaciones):
    data = []
    # Buscar la fila correspondiente al número de frame
    row = df[df['frame_number'] == frame_number]

    for articulacion in articulaciones:
        x = row[articulacion+ '_X'].iloc[0]
        y = row[articulacion+ '_Y'].iloc[0]
        z = row[articulacion+ '_Z'].iloc[0]

        data.append([x, y, z])
    return data

# Función para extraer velocidad

Dado un dataframe y un numero de frame, retorna la velocidad instantánea correspondiente a la fila con el numero de frame pasado por parámetro.

In [9]:
def extraer_velocidad(df, frame_number):
    # Buscar la fila correspondiente al número de frame
    row = df[df['frame_number'] == frame_number]
    return row["Velocidad(Cadera)"].iloc[0]

#Función para convertir coordenadas normalizadas a metros

In [10]:
def coordenadas_a_distancia(a, b):
    # Calcula la diferencia en coordenadas normalizadas
    dx = b[0] - a[0]
    dy = b[1] - a[1]

    # Relación pixel-distancia en el eje x y en el eje y
    # primer coordenada: proporción de x
    # segunda coordenada: porporcion de y
    relacion_pixel_distancia = (0.58/0.292, 0.614)

    # Convierte la diferencia en píxeles a distancia real usando la relación pixel-distancia
    distancia_x = dx * relacion_pixel_distancia[0]
    distancia_y = dy * relacion_pixel_distancia[1]

    # Calcula la distancia euclidiana en el mundo real
    return (distancia_x**2 + distancia_y**2)**0.5


# Velocidad angular
Calcula la velocidad angular entre tres puntos a, b y c


Retorna la velocidad angular en radianes por segundo.

La velocidad angular (ω) es una medida de la rapidez con la que un objeto rota alrededor de un eje específico.

Se define como el cambio en el ángulo (θ) que recorre el objeto por unidad de tiempo.

In [11]:
def velocidad_angular(angulo_inicial, angulo_final, delta_tiempo):
    # Calcular el cambio en el ángulo
    delta_theta = angulo_final - angulo_inicial

    # Calcular la velocidad angular
    # Recordar que omega = theta punto = vel angular = Delta theta / Delta t
    # Donde Delta theta es el cambio en rotación angular y Delta t es el cambio en el tiempo
    # ENTONCES: la velocidad angular se calcula dividiendo la diferencia total del ángulo (delta_theta) por el tiempo transcurrido entre las mediciones (1 / frame_rate).
    angular_velocity = delta_theta / delta_tiempo
    return angular_velocity

# Funcion para obtener Centro de Masa

---


Es el punto donde se puede considerar que toda la masa del objeto está distribuida uniformemente.

Centro de masa = (sumatoria de las masas * sumatoria de las posiciones)/masa total




In [12]:
def obtenerCentroDeMasa():
   # suponiendo que la persona pesa 65kg y la barra viendo los discos pesa otros 60kg
    masaTotal = 125 # en kilos
    SumatoriaX = 0
    SumatoriaY = 0

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        for landmark in mp_pose.PoseLandmark:
            SumatoriaX += masaTotal*results.pose_landmarks.landmark[landmark].x
            SumatoriaY += masaTotal*results.pose_landmarks.landmark[landmark].y


    SumatoriaX /= masaTotal
    SumatoriaY /= masaTotal

    # preguntar si lo que vamos a retornar nos va a dar la posicion de un frame o que nos de una coordenada
    # deberia dar coordenada
    return (SumatoriaX , SumatoriaY)

#Dibujar diagrama

In [13]:
def diagrama_cuerpo(frame_number):
  #centro = obtenerCentroDeMasa()
  left_wrist,right_wrist,left_ankle =  extraer_posiciones(df_completo,frame_number,'LEFT_WRIST','RIGHT_WRIST','LEFT_ANKLE')
  print()
  centro = ( (left_wrist[0] + right_wrist[0]) / 2 , (left_wrist[1] + left_ankle[1]) / 2 )
  cv2.circle(image, (int(centro[0] * video_width) , int(centro[1] * video_height)) , 20, (255,0,255), -1,3)
  # Grafico Peso
  cv2.arrowedLine(image, (int(centro[0] * video_width) , int(centro[1] * video_height)) , (int(centro[0]* video_width) , int(centro[1]* video_height + video_height/6) ) , (255,0,0), 4)
  # Grafico normal
  cv2.arrowedLine(image, (int(centro[0] * video_width) , int(centro[1] * video_height)) , (int(centro[0]* video_width) , int(centro[1]* video_height - video_height/6) ) , (255,0,0), 4)
  textoPeso =  str(peso_persona + peso_pesa)+" Peso"
  textoNormal =  str(peso_persona + peso_pesa)+" Normal"

  cv2.putText(image, textoPeso, (int(centro[0]* video_width) , int(centro[1]* video_height + video_height/6) ), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4, cv2.LINE_AA)
  cv2.putText(image, textoPeso, (int(centro[0]* video_width) , int(centro[1]* video_height - video_height/6) ), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4, cv2.LINE_AA)
  #cv2.line(image, int(centro[0]) * video_width , int(centro[0]-20) * video_width , (255,0,0), 4) #peso



# Diagrama de cuerpo libre / Peso, Normal y Fuerza


In [14]:
import matplotlib.pyplot as plt

def diagramaDeCuerpo():
    # Valores fijos se encuentran declarados al inicio
    # Valor de la gravedad / aceleracion m/s2
    g = 9.81
    # Peso del cuerpo = 637
    peso_cuerpo = masa_persona * g
    # Peso de la barra / Evaluamos la barra y sus discos como un cuerpo = 1372N
    peso_barra = masa_pesa * g

    # Normal de la barra = 1372N
    normal_barra = peso_barra
    # Normal del cuerpo = 2009
    normal_cuerpo = normal_barra + peso_cuerpo

    # Tomaremos un valor estático para simular la fuerza realizada por la persona para levantar la barra. Se aplica sobre el cuerpo
    # Segunda ley de Newton: sumatoria de fuerzas = masa * aceleracion. La fuerza debe ser mayor que 1372N
    fuerza_empuje = 1400

    if normal_cuerpo > 0:
        normal_direccion_cuerpo = "Upward"
        peso_direccion_cuerpo = "Upward"
    else:
        normal_direccion_cuerpo = "Downward"
        peso_direccion_cuerpo = "Downward"

    if normal_barra > 0:
        normal_direccion_barra = "Upward"
        peso_direccion_barra = "Upward"
    else:
        normal_direccion_barra = "Downward"
        peso_direccion_barra = "Downward"

    if fuerza_empuje > 0:
        fuerza_direccion_cuerpo = "Upward"
    else:
        fuerza_direccion_cuerpo = "Downward"

    return normal_direccion_cuerpo, peso_direccion_cuerpo, normal_direccion_barra, peso_direccion_barra, fuerza_direccion_cuerpo

def generate_free_body_diagrams():
    # Figura y ejes
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    # Diagrama de cuerpo libre de la barra
    axs[0].arrow(0, 0, 0, -1372, head_width=50, head_length=100, fc='red', ec='red')
    axs[0].arrow(0, 0, 0, -637, head_width=50, head_length=100, fc='blue', ec='blue')
    axs[0].set_xlim(-2000, 2000)
    axs[0].set_ylim(-2000, 2000)
    axs[0].set_xlabel('X')
    axs[0].set_ylabel('Y')
    axs[0].set_title('Diagrama de Cuerpo libre - Barra')

    # Diagrama de cuerpo libre para el cuerpo
    axs[1].arrow(0, 0, 0, -2009, head_width=50, head_length=100, fc='red', ec='red')
    axs[1].arrow(0, 0, 0, -1400, head_width=50, head_length=100, fc='green', ec='green')
    axs[1].arrow(0, 0, 0, -637, head_width=50, head_length=100, fc='blue', ec='blue')
    axs[1].set_xlim(-2000, 2000)
    axs[1].set_ylim(-2000, 2000)
    axs[1].set_xlabel('X')
    axs[1].set_ylabel('Y')
    axs[1].set_title('Diagrama de cuerpo libre - Cuerpo')

    # Diagrama de cuerpo libre del cuerpo con la barra encima
    axs[2].arrow(0, 0, 0, -2009, head_width=50, head_length=100, fc='red', ec='red')
    axs[2].arrow(0, 0, 0, -2772, head_width=50, head_length=100, fc='green', ec='green')
    axs[2].arrow(0, 0, 0, -1372, head_width=50, head_length=100, fc='blue', ec='blue')
    axs[2].arrow(0, -1372, 0, -637, head_width=50, head_length=100, fc='blue', ec='blue')
    axs[2].set_xlim(-2000, 2000)
    axs[2].set_ylim(-4000, 0)
    axs[2].set_xlabel('X')
    axs[2].set_ylabel('Y')
    axs[2].set_title('Diagrama de cuerpo libre - Cuerpo con la barra')

    plt.tight_layout()
    plt.show()

#Funciones para ejecutar el codigo
diagramaDeCuerpo()
generate_free_body_diagrams()

NameError: name 'masa_persona' is not defined

#Vector Velocidad instantanea

La velocidad instantánea es la velocidad de un objeto en un momento específico en el tiempo.

Se calcula como v(t) = dx / dt



In [15]:
def velocidad_instantanea(pos_anterior, pos_actual, tiempo):
  dx = pos_actual[0] - pos_anterior[0]
  dy = pos_actual[1] - pos_anterior[1]
  return (dx/tiempo, dy/tiempo)

#Vector Aceleracion instantanea
La aceleración instantánea es la tasa de cambio instantánea de la velocidad de un objeto en un punto específico de su trayectoria.

a(t) = dv / dt


In [16]:
def aceleracion_instantanea(vel_actual_x, vel_anterior_x, vel_actual_y, vel_anterior_y, tiempo):
  dvx = vel_actual_x - vel_anterior_x
  dvy = vel_actual_y - vel_anterior_y
  return (dvx/tiempo, dvy/tiempo)
#df_completo["frame_number"] == frame_number, "Velocidad(Cadera)"]

#Cálculo de Torque

Se analiza los torques de cadera y rodilla durante una sentadilla. Primero, identifica las posiciones de las articulaciones de la cadera, rodilla y tobillo.

Luego, calcula las distancias y ángulos entre las articulaciones, y utiliza estas medidas para estimar el torque generado por cada articulación.

TORQUE = F x brazo momento

Brazo momento = distancia entre articulación y carga (es la distancia desde el punto donde se aplica una fuerza hasta el punto de giro, en este caso el punto de giro es la cadera y la rodilla, y el punto de fuerza se hace desde la muñeca)
F = 2/3 de lo que pesa la persona (cabeza y torso) + peso de carga

In [17]:
def calcular_torques(a, b, c, d):
   # left_wrist, left_hip, left_knee, left_ankle = extraer_posiciones(landmarks,frame_number ,'LEFT_WRIST', 'LEFT_HIP', 'LEFT_KNEE', 'LEFT_ANKLE')
    e = (a[0], b[1])
    r = (a[0], c[1])
    #distancia_a_b = coordenadas_a_distancia(a, b)
    #distancia_b_c = coordenadas_a_distancia(b, c)
    #distancia_c_d = coordenadas_a_distancia(c, d)
    # Perpendicular a la fuerza
    distancia_b_e = coordenadas_a_distancia(b, e)
    distancia_c_r = coordenadas_a_distancia(c, r)


    #angulo_a_b_c = calculate_angle(a, b, c)
    #angulo_a_b_e = calculate_angle(a, b, e)

    # La biomecanica de la rodilla funciona al revez que en el resto del cuerpo.
    #angulo_b_c_d = 180 - calculate_angle(b, c, d)
    # print("angulo de 90:", calculate_angle(a,e,b))
    # print("distancia_b_e ", distancia_b_e)
    # print("(a[0] - b[0]) ", (a[0] - b[0]) * video_width)
    # print("brazo calculado con angulo", distancia_a_b * math.cos(calculate_angle(a,b,e)))

    start_point_torque_cadera = (int(b[0] * video_width), int(b[1] * video_height))
    end_point_torque_cadera = (int(e[0] * video_width), int(e[1] * video_height))
    cv2.line(image, start_point_torque_cadera, end_point_torque_cadera,(0,255,0), 3)

    start_point_torque_rodilla = (int(c[0] * video_width), int(c[1] * video_height))
    end_point_torque_rodilla = (int(r[0] * video_width), int(r[1] * video_height))
    cv2.line(image, start_point_torque_rodilla, end_point_torque_rodilla,(0,255,255), 3)
    # Torque en N.m
    torque_cadera = (peso_persona * 2/3 + peso_pesa) * distancia_b_e
    torque_rodilla = (peso_persona * 2/3 + peso_pesa) * distancia_c_r
    return [torque_rodilla, torque_cadera]
    # Comparar resultados de los torques y mostrar algo en el video

# Visualizar trackeo articulaciones relevantes

In [18]:
def dibujar_articulaciones_relevantes(articulacion1,articulacion2,articulacion3,articulacion4):
    art1 = np.array(articulacion1);
    art2 = np.array(articulacion2);
    art3 = np.array(articulacion3);
    art4 = np.array(articulacion4);
    angle = calculate_angle(articulacion1,articulacion2,articulacion3)
    color = (0, 0, 255)
    #Dibujo el primer tramo de la pierna
    i1 = (int(art1[0] * video_width), int(art1[1] * video_height))
    f1 = (int(art2[0] * video_width), int(art2[1] * video_height))
    cv2.line(image, i1, f1, color, 5)

    #Dibujo el segundo tramo de la pierna
    i2 = (int(art2[0] * video_width), int(art2[1] * video_height))
    f2 = (int(art3[0] * video_width), int(art3[1] * video_height))
    cv2.line(image, i2, f2, color, 5)

    i3 = (int(art3[0] * video_width), int(art3[1] * video_height))
    f3 = (int(art4[0] * video_width), int(art4[1] * video_height))
    cv2.line(image, i3, f3, color, 5)

#Visualizar esfuerzo piernas

In [19]:
def dibujar_piernas(articulacion1,articulacion2,articulacion3):
    art1 = np.array(articulacion1);
    art2 = np.array(articulacion2);
    art3 = np.array(articulacion3);
    #angle = calculate_angle(articulacion1,articulacion2,articulacion3)

    torques = calcular_torques(pos_actual_wrist,pos_actual_left_hip,pos_actual_left_knee,pos_actual_left_ankle)
    df_completo.loc[df_completo["frame_number"] == frame_number, "Torque(Rodilla)"] = torques[0]
    df_completo.loc[df_completo["frame_number"] == frame_number, "Torque(Cadera)"] = torques[1]
    #Dibujo el primer tramo de la pierna
    centro_art2 = (int(art2[0] * video_width), int(art2[1] * video_height))
    centro_art3 = (int(art3[0] * video_width), int(art3[1] * video_height))
    #f1 = (int(art2[0] * video_width), int(art2[1] * video_height))

    if(torques[0] >= torques[1] * 3/4):
      cv2.circle(image, centro_art2,20, (255,0,0), -1,3)
      cv2.circle(image, centro_art3,20, (255,0,0), -1,3)
    else:
      if(torques[1] * 1/4 < torques[0] < torques[1] * 3/4):
        color = (255,255,0)
        cv2.circle(image, centro_art2,20, (255,0,0), -1,3)
        cv2.circle(image, centro_art3,20, (255,255,0), -1,3)
      else:
        cv2.circle(image, centro_art2,20, (255,0,0), -1,3)
        cv2.circle(image, centro_art3,20, (0,255,0), -1,3)

    # Dibujo el segundo tramo de la pierna
    # f2 = (int(art3[0] * video_width), int(art3[1] * video_height))
    # cv2.circle(image, centro_art3,20, color, 3, 20)

#**Código para recorrer frames del video y realizar cálculos**

Este bloque de código recorre cada frame del video, procesa la imagen utilizando MediaPipe para detectar landmarks de la pose, y guarda los datos en un DataFrame. Luego, calcula el ángulo entre las articulaciones de la cadera, la rodilla y el tobillo. Después, dibuja los landmarks detectados en el video y guarda el video procesado en el archivo de salida. Finalmente, libera los recursos utilizados y guarda los datos de la pose en un archivo CSV.

In [ ]:
# Recorro todos los frames del video
frame_number = 0

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Convertir la imagen a RGB (el fotograma)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Procesar la imagen con MediaPipe y guardar los resultados
    results = pose.process(image)
    # Recolectar y guardar los datos de la pose en el dataframe
    pose_row = {'frame_number': frame_number}

    # Extraer posiciones
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        #Por cada articulacion, guarda en su posicion de X, Y, Z el resultado
        for landmark in articulaciones:
            pose_row[landmark.name + '_X'] = results.pose_landmarks.landmark[landmark].x
            pose_row[landmark.name + '_Y'] = results.pose_landmarks.landmark[landmark].y
            pose_row[landmark.name + '_Z'] = results.pose_landmarks.landmark[landmark].z
    else:
        ##for landmark in mp_pose.PoseLandmark:
        for landmark in articulaciones:
            pose_row[landmark.name + '_X'] = None
            pose_row[landmark.name + '_Y'] = None
            pose_row[landmark.name + '_Z'] = None

    pose_row_df = pd.DataFrame(pose_row, index = [pose_row['frame_number']])
    df_completo = pd.concat([df_completo, pose_row_df], ignore_index = True)

    # Agrego los landmarks al gráfico
   # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
    #                          mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=5, circle_radius=5),
   #                           mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=5, circle_radius=5)
    #                          )

    if(frame_number>0):
      # Crear una lista de tiempo para cada cuadro del video
      # df_completo.append("Tiempo")
      # Agregar la lista de tiempo como una columna al DataFrame
      # fila = df_completo[df_completo['frame_number'] == frame_number]
      # fila["Tiempo"] = tiempo_por_frame * frame_number  #df_completo['Tiempo'] = tiempo_por_frame * frame_number
      df_completo.loc[df_completo["frame_number"] == frame_number, "Tiempo"] = tiempo_por_frame * frame_number
      # print("tiempo en cada frame",tiempo_por_frame * frame_number)
      previous_frame = frame_number - 1

      pos_prev_left_hip, pos_prev_left_knee, pos_prev_left_ankle = extraer_posiciones(df_completo, previous_frame, 'RIGHT_HIP', 'RIGHT_KNEE', 'RIGHT_ANKLE')

      pos_actual_wrist, pos_actual_left_hip, pos_actual_left_knee, pos_actual_left_ankle = extraer_posiciones(df_completo, frame_number,'RIGHT_WRIST', 'RIGHT_HIP', 'RIGHT_KNEE', 'RIGHT_ANKLE')
      dibujar_articulaciones_relevantes(pos_actual_wrist,pos_actual_left_hip,pos_actual_left_knee,pos_actual_left_ankle)
      diagrama_cuerpo(frame_number)
      #---------------VELOCIDAD ANGULAR------------
      angulo_anterior = calculate_angle((pos_prev_left_hip[0], pos_prev_left_hip[1]), (pos_prev_left_knee[0], pos_prev_left_knee[1]), (pos_prev_left_ankle[0], pos_prev_left_ankle[1]))
      angulo_actual = calculate_angle((pos_actual_left_hip[0], pos_actual_left_hip[1]), (pos_actual_left_knee[0], pos_actual_left_knee[1]), (pos_actual_left_ankle[0], pos_actual_left_ankle[1]))
      vel_angular = velocidad_angular(angulo_anterior, angulo_actual, tiempo_por_frame)
      # print("angulo anterior en frame: ",angulo_anterior, " numero: ",frame_number)
      # print("angulo angulo_actual",angulo_actual)
      df_completo.loc[df_completo["frame_number"] == frame_number, "VelocidadAngular"] = vel_angular

      #---------------Velocidad instantanea---------
      velocidad_cadera = velocidad_instantanea(pos_prev_left_hip, pos_actual_left_hip, tiempo_por_frame)
      velocidad_rodilla = velocidad_instantanea(pos_prev_left_knee, pos_actual_left_knee, tiempo_por_frame)
      #velocidad_cadera_str = ','.join(map(str, velocidad_cadera))
      #velocidad_rodilla_str = ','.join(map(str, velocidad_rodilla))
      df_completo.loc[df_completo["frame_number"] == frame_number, "Velocidad(Cadera)_X"] = velocidad_cadera[0]
      df_completo.loc[df_completo["frame_number"] == frame_number, "Velocidad(Cadera)_Y"] = velocidad_cadera[1]
      df_completo.loc[df_completo["frame_number"] == frame_number, "Velocidad(Rodilla)_X"] = velocidad_rodilla[0]
      df_completo.loc[df_completo["frame_number"] == frame_number, "Velocidad(Rodilla)_Y"] = velocidad_rodilla[1]

      #---------------Aceleracion instantanea---------
      aceleracion_actual_cadera = aceleracion_instantanea(df_completo.loc[df_completo["frame_number"], "Velocidad(Cadera)_X"].iloc[0], df_completo.loc[df_completo["frame_number"] == frame_number - 1, "Velocidad(Cadera)_X"].iloc[0],df_completo.loc[df_completo["frame_number"], "Velocidad(Cadera)_Y"].iloc[0], df_completo.loc[df_completo["frame_number"] == frame_number - 1, "Velocidad(Cadera)_Y"].iloc[0],tiempo_por_frame)
      aceleracion_actual_rodilla = aceleracion_instantanea(df_completo.loc[df_completo["frame_number"], "Velocidad(Rodilla)_X"].iloc[0], df_completo.loc[df_completo["frame_number"] == frame_number - 1, "Velocidad(Rodilla)_X"].iloc[0],df_completo.loc[df_completo["frame_number"], "Velocidad(Rodilla)_Y"].iloc[0], df_completo.loc[df_completo["frame_number"] == frame_number - 1, "Velocidad(Rodilla)_Y"].iloc[0],tiempo_por_frame)
      #aceleracion_actual_rodilla = aceleracion_instantanea(df_completo.loc[df_completo["frame_number"], "Velocidad(Rodilla)_X"].iloc[0], df_completo.loc[df_completo["frame_number"] == frame_number - 1, "Velocidad(Rodilla)"].iloc[0], tiempo_por_frame)
      #print("aceleracion_actual_cadera",df_completo.loc[df_completo["frame_number"], "Velocidad(Cadera)_X"].iloc[0])
      #print("aceleracion_actual_rodilla",aceleracion_actual_rodilla)

      # velocidad_actual_cadera =
      # aceleracion_cadera = aceleracion_instantanea()
      df_completo.loc[df_completo["frame_number"] == frame_number, "Aceleracion(Rodilla)_X"] = aceleracion_actual_rodilla[0]
      df_completo.loc[df_completo["frame_number"] == frame_number, "Aceleracion(Rodilla)_Y"] = aceleracion_actual_rodilla[1]
      df_completo.loc[df_completo["frame_number"] == frame_number, "Aceleracion(Cadera)_X"] = aceleracion_actual_cadera[0]
      df_completo.loc[df_completo["frame_number"] == frame_number, "Aceleracion(Cadera)_Y"] = aceleracion_actual_cadera[1]

      #--------------Torque------------------------
      #torques = calcular_torques(pos_actual_wrist,pos_actual_left_hip,pos_actual_left_knee,pos_actual_left_ankle)
      #df_completo.loc[df_completo["frame_number"] == frame_number, "Torque(Rodilla)"] = torques[0]
      #df_completo.loc[df_completo["frame_number"] == frame_number, "Torque(Cadera)"] = torques[1]
      dibujar_piernas(pos_actual_wrist,pos_actual_left_hip,pos_actual_left_knee)

    else:
        df_completo.loc[df_completo["frame_number"] == frame_number, "Tiempo"] = 0
        df_completo.loc[df_completo["frame_number"] == frame_number, "VelocidadAngular"] = 0
        df_completo.loc[df_completo["frame_number"] == frame_number, "Velocidad(Cadera)_X"] = 0
        df_completo.loc[df_completo["frame_number"] == frame_number, "Velocidad(Cadera)_Y"] = 0
        df_completo.loc[df_completo["frame_number"] == frame_number, "Velocidad(Rodilla)_X"] = 0
        df_completo.loc[df_completo["frame_number"] == frame_number, "Velocidad(Rodilla)_Y"] = 0
    # Guardar el cuadro procesado en el video de salida
    video_writer.write(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

    frame_number += 1

if pose._graph is not None:
    pose.close()

video_writer.release()
cap.release()

# Guardar el dataframe como CSV
df_completo.to_csv(output_csv_path, index=False)

df = pd.read_csv(output_csv_path, index_col=[0]) # Leo los datos del csv
pd.options.display.max_columns = None

print("Proceso completado. Video trackeado guardado en:", output_video_path)
print("Datos de la pose guardados en:", output_csv_path)

In [ ]:
df_completo

#Graficos

In [35]:
#-------------------VERSION CON PLOTLY---------------------------

window_size = 50
left_hip_y_smoothed = df_completo['LEFT_HIP_Y'].rolling(window=window_size).mean()
left_knee_y_smoothed = df_completo['LEFT_KNEE_Y'].rolling(window=window_size).mean()

#------------POSICIONES DE CADERA Y RODILLA----------------------
trace1 = go.Scatter(x=df_completo.index, y=left_hip_y_smoothed, mode='lines', name='Altura de la cadera', line=dict(color='blue'))
trace2 = go.Scatter(x=df_completo.index, y=left_knee_y_smoothed, mode='lines', name='Posición de la rodilla', line=dict(color='red'))

fig_posiciones = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.1)
fig_posiciones.add_trace(trace1, row=1, col=1)
fig_posiciones.add_trace(trace2, row=1, col=1)

fig_posiciones.update_layout(
    title='Evolución de la posición de la cadera y la rodilla con respecto al tiempo',
    xaxis=dict(title='Tiempo'),
    yaxis=dict(title='Posición', autorange='reversed'),  # Invertir eje Y
    yaxis2=dict(title='Posición', autorange='reversed'),  # Invertir eje Y
    legend=dict(x=0.7, y=1.1),
    height=600,
    width=800
)


#------------VELOCIDAD----------------------
# Crear figura para la velocidad de la cadera
fig_velocidad = px.line(df_completo, x='Tiempo', y='Velocidad(Cadera)_Y', title='Evolución de la velocidad de la cadera')
fig_velocidad.update_xaxes(title_text='Tiempo')
fig_velocidad.update_yaxes(title_text='Velocidad de la cadera')


#-----------ACELERACIÓN---------------------
# Crear figura para la aceleración de la cadera
fig_aceleracion = px.line(df_completo, x='Tiempo', y='Aceleracion(Cadera)_Y', title='Evolución de la aceleración de la cadera')
fig_aceleracion.update_xaxes(title_text='Tiempo')
fig_aceleracion.update_yaxes(title_text='Aceleración de la cadera')

# Mostrar las figuras
fig_posiciones.show()
fig_velocidad.show()
fig_aceleracion.show()

In [ ]:
coordenadas_a_distancia((0.57, 0.322), (0.66, 0.293))

0.1796517130576536